# Biot Poroelasticity Visualization Hub

**Comprehensive visualization and validation notebook for 2D Biot poroelasticity physics-informed neural networks**

This notebook serves as the central hub for visualizing and validating all aspects of the Biot poroelasticity project:

## Contents Overview
1. **Environment Setup & Imports** - Import libraries and check dependencies
2. **Physics-Only Trainer Validation** - Validate core physics implementation
3. **Data-Enhanced Training** - Integrate experimental VTK data
4. **Comparative Analysis** - Compare physics-only vs data-enhanced approaches
5. **Interactive Parameter Studies** - Sensitivity analysis and optimization
6. **Future Extensions** - 3D visualization and advanced capabilities

---

**Note:** This notebook is designed to work with your existing Python environment without conflicts. All visualizations are self-contained and modular.

## Environment Setup & Imports

Import all necessary libraries and check if the custom modules are available.

In [ ]:
# Standard library imports
import sys
import os
import warnings
from pathlib import Path

# Add parent directory to Python path for importing our modules
# Since we're in poroelasticity/notebooks/, go up one level to poroelasticity/
parent_dir = Path(os.getcwd()).parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Also add the FBPINNs root directory (two levels up)
fbpinns_root = parent_dir.parent
if str(fbpinns_root) not in sys.path:
    sys.path.insert(0, str(fbpinns_root))

print(f"Working directory: {os.getcwd()}")
print(f"Python paths added:")
print(f"  - {parent_dir}")
print(f"  - {fbpinns_root}")

# Scientific computing imports
try:
    import numpy as np
    import jax.numpy as jnp
    import jax
    print("SUCCESS: JAX libraries loaded")
    jax_available = True
except ImportError as e:
    print(f"WARNING: JAX not available: {e}")
    import numpy as np
    print("SUCCESS: NumPy fallback loaded")
    jax_available = False

# Visualization imports
try:
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from matplotlib.animation import FuncAnimation
    print("SUCCESS: Matplotlib loaded")
    plotting_available = True
except ImportError as e:
    print(f"WARNING: Matplotlib not available: {e}")
    plotting_available = False

# Optional advanced visualization
try:
    from mpl_toolkits.mplot3d import Axes3D
    print("SUCCESS: 3D plotting available")
    plotting_3d_available = True
except ImportError:
    plotting_3d_available = False

print("\n" + "="*50)
print("ENVIRONMENT STATUS:")
print(f"JAX: {'Available' if jax_available else 'Not available (using NumPy)'}")
print(f"Plotting: {'Available' if plotting_available else 'Not available'}")
print(f"3D Plotting: {'Available' if plotting_3d_available else 'Not available'}")
print("="*50)

## Module Loading and Validation

Load the Biot trainer modules and check their availability.

In [ ]:
# Import Biot trainer modules
print("Loading Biot trainer modules...")

# Status tracking
module_status = {}

# Physics-only trainer
try:
    from trainers.biot_trainer_2d import BiotCoupledTrainer, BiotCoupled2D, CoupledTrainer
    print("SUCCESS: Physics-only trainer loaded")
    module_status['physics'] = True
except ImportError as e:
    print(f"ERROR: Physics trainer not available: {e}")
    module_status['physics'] = False

# Data-enhanced trainer
try:
    from trainers.biot_trainer_2d_data import BiotCoupledDataTrainer, VTKDataLoader, BiotCoupled2DData
    print("SUCCESS: Data-enhanced trainer loaded")
    module_status['data'] = True
except ImportError as e:
    print(f"WARNING: Data trainer not available: {e}")
    module_status['data'] = False

# Test imports that we know work from your testing
try:
    import fbpinns
    print("SUCCESS: FBPINNs core library loaded")
    module_status['fbpinns'] = True
except ImportError as e:
    print(f"WARNING: FBPINNs not available: {e}")
    module_status['fbpinns'] = False

# Summary
print("\n" + "="*50)
print("MODULE STATUS:")
if module_status['physics']:
    print("CORE: Physics-only training ready")
if module_status['data']:
    print("DATA: Data-enhanced training ready")
if module_status['fbpinns']:
    print("LIB: FBPINNs library ready")

if not module_status['physics']:
    print("ERROR: Critical physics module missing")
    print("       Make sure you're in the correct directory")

print("\nQUICK START:")
print("1. Run a physics validation test")
print("2. Visualize solution fields")
print("3. Analyze error metrics")
print("="*50)

## Quick Physics Validation

Test the physics-only trainer with minimal training to ensure everything works.

In [ ]:
def quick_physics_test():
    """Run a quick physics validation test"""
    if not module_status.get('physics', False):
        print("ERROR: Physics trainer not available")
        return None
    
    print("Starting quick physics validation...")
    
    try:
        # Create trainer with correct parameters
        trainer = BiotCoupledTrainer(
            w_mech=1.0,     # Weight for mechanics equations
            w_flow=1.0,     # Weight for flow equations  
            w_bc=1.0,       # Weight for boundary conditions
            auto_balance=True  # Use automatic loss balancing
        )
        
        print("SUCCESS: Trainer created")
        
        # Quick training with gradual coupling
        print("Running quick training with gradual coupling...")
        trainer.train_gradual_coupling(n_steps_pre=25, n_steps_coupled=50)
        
        print("SUCCESS: Quick training completed")
        
        # Get final loss from the underlying trainer
        try:
            final_loss = trainer.trainer.test_loss()
            print(f"Final test loss: {final_loss:.6f}")
        except:
            print("Test loss not available, but training completed successfully!")
        
        return trainer
        
    except Exception as e:
        print(f"ERROR in quick test: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the test
test_trainer = quick_physics_test()

## Solution Field Visualization

Visualize the displacement and pressure fields from the trained model.

In [ ]:
def plot_biot_solution(trainer, nx=30, ny=30, figsize=(15, 10)):
    """Plot Biot poroelasticity solution fields"""
    if trainer is None:
        print("No trainer provided for visualization")
        return
    
    if not plotting_available:
        print("Matplotlib not available for plotting")
        return
    
    print(f"Creating solution visualization ({nx}x{ny} grid)...")
    
    # Create mesh grid
    x = np.linspace(0, 1, nx)
    y = np.linspace(0, 1, ny)
    X, Y = np.meshgrid(x, y)
    
    # Flatten for prediction
    x_flat = X.flatten()
    y_flat = Y.flatten()
    points = np.column_stack([x_flat, y_flat])
    
    try:
        # Get predictions
        if jax_available:
            points_input = jnp.array(points)
        else:
            points_input = points
        
        # Predict solution
        pred = trainer.predict(points_input)
        
        # Convert to numpy if needed
        if hasattr(pred, 'numpy'):
            pred = pred.numpy()
        elif jax_available and hasattr(pred, '__array__'):
            pred = np.array(pred)
        
        # Get exact solution for comparison
        try:
            exact = trainer.trainer.c.problem.exact_solution(trainer.all_params, points_input)
            if hasattr(exact, 'numpy'):
                exact = exact.numpy()
            elif jax_available and hasattr(exact, '__array__'):
                exact = np.array(exact)
            has_exact = True
        except:
            print("Warning: Exact solution not available")
            has_exact = False
        
        # Reshape for plotting
        ux_pred = pred[:, 0].reshape(X.shape)
        uy_pred = pred[:, 1].reshape(X.shape)
        p_pred = pred[:, 2].reshape(X.shape)
        
        # Create plots
        if has_exact:
            fig, axes = plt.subplots(2, 3, figsize=figsize)
            
            ux_exact = exact[:, 0].reshape(X.shape)
            uy_exact = exact[:, 1].reshape(X.shape)
            p_exact = exact[:, 2].reshape(X.shape)
            
            # Top row: Predicted
            im1 = axes[0, 0].contourf(X, Y, ux_pred, levels=20, cmap='RdBu_r')
            axes[0, 0].set_title('$u_x$ (Predicted)')
            plt.colorbar(im1, ax=axes[0, 0])
            
            im2 = axes[0, 1].contourf(X, Y, uy_pred, levels=20, cmap='RdBu_r')
            axes[0, 1].set_title('$u_y$ (Predicted)')
            plt.colorbar(im2, ax=axes[0, 1])
            
            im3 = axes[0, 2].contourf(X, Y, p_pred, levels=20, cmap='viridis')
            axes[0, 2].set_title('Pressure $p$ (Predicted)')
            plt.colorbar(im3, ax=axes[0, 2])
            
            # Bottom row: Exact
            im4 = axes[1, 0].contourf(X, Y, ux_exact, levels=20, cmap='RdBu_r')
            axes[1, 0].set_title('$u_x$ (Exact)')
            plt.colorbar(im4, ax=axes[1, 0])
            
            im5 = axes[1, 1].contourf(X, Y, uy_exact, levels=20, cmap='RdBu_r')
            axes[1, 1].set_title('$u_y$ (Exact)')
            plt.colorbar(im5, ax=axes[1, 1])
            
            im6 = axes[1, 2].contourf(X, Y, p_exact, levels=20, cmap='viridis')
            axes[1, 2].set_title('Pressure $p$ (Exact)')
            plt.colorbar(im6, ax=axes[1, 2])
            
            fig.suptitle('Biot Poroelasticity: Predicted vs Exact Solution', fontsize=16)
            
        else:
            # Just show predicted solution
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            im1 = axes[0].contourf(X, Y, ux_pred, levels=20, cmap='RdBu_r')
            axes[0].set_title('$u_x$ (Predicted)')
            plt.colorbar(im1, ax=axes[0])
            
            im2 = axes[1].contourf(X, Y, uy_pred, levels=20, cmap='RdBu_r')
            axes[1].set_title('$u_y$ (Predicted)')
            plt.colorbar(im2, ax=axes[1])
            
            im3 = axes[2].contourf(X, Y, p_pred, levels=20, cmap='viridis')
            axes[2].set_title('Pressure $p$ (Predicted)')
            plt.colorbar(im3, ax=axes[2])
            
            fig.suptitle('Biot Poroelasticity Solution Fields', fontsize=16)
        
        plt.tight_layout()
        plt.show()
        
        # Print some statistics
        print("\nSolution Statistics:")
        print(f"u_x range: [{ux_pred.min():.4f}, {ux_pred.max():.4f}]")
        print(f"u_y range: [{uy_pred.min():.4f}, {uy_pred.max():.4f}]")
        print(f"p range: [{p_pred.min():.4f}, {p_pred.max():.4f}]")
        
        if has_exact:
            # Calculate errors
            ux_error = np.mean((ux_pred - ux_exact)**2)**0.5
            uy_error = np.mean((uy_pred - uy_exact)**2)**0.5
            p_error = np.mean((p_pred - p_exact)**2)**0.5
            
            print("\nL2 Errors:")
            print(f"u_x error: {ux_error:.6f}")
            print(f"u_y error: {uy_error:.6f}")
            print(f"p error: {p_error:.6f}")
            
    except Exception as e:
        print(f"Error in visualization: {e}")
        import traceback
        traceback.print_exc()

# Visualize the test trainer if available
if test_trainer is not None:
    plot_biot_solution(test_trainer)
else:
    print("No trained model available for visualization")
    print("Run the quick physics test first")

## Physics Accuracy Testing & Model Diagnostics

Test the physics accuracy and diagnose learning issues when the model isn't performing well.

In [ ]:
def comprehensive_physics_diagnostics(trainer):
    """Comprehensive diagnostics to identify why the model isn't learning properly"""
    if trainer is None:
        print("❌ No trainer provided for diagnostics")
        return
    
    print("🔍 COMPREHENSIVE PHYSICS DIAGNOSTICS")
    print("="*60)
    
    # 1. Check if trainer has the required components
    print("\n1. TRAINER STRUCTURE CHECK:")
    print(f"   Trainer type: {type(trainer).__name__}")
    print(f"   Has underlying trainer: {hasattr(trainer, 'trainer')}")
    
    if hasattr(trainer, 'trainer'):
        print(f"   Underlying trainer type: {type(trainer.trainer).__name__}")
        print(f"   Has constants: {hasattr(trainer.trainer, 'c')}")
        print(f"   Has parameters: {hasattr(trainer.trainer, 'all_params')}")
        
        # Access the underlying trainer
        base_trainer = trainer.trainer
        
        # 2. Check loss components
        print("\n2. LOSS COMPONENT ANALYSIS:")
        if hasattr(base_trainer, 'loss_log') and len(base_trainer.loss_log) > 0:
            latest_losses = base_trainer.loss_log[-1]
            print(f"   Latest total loss: {latest_losses.get('loss', 'N/A')}")
            print(f"   PDE loss: {latest_losses.get('loss_pde', 'N/A')}")
            print(f"   Boundary loss: {latest_losses.get('loss_boundary', 'N/A')}")
            print(f"   Data loss: {latest_losses.get('loss_data', 'N/A')}")
        else:
            print("   ⚠️ No detailed loss history available")
        
        # 3. Check parameters and gradients
        print("\n3. PARAMETER CHECK:")
        if hasattr(trainer, 'all_params') or hasattr(base_trainer, 'all_params'):
            params = getattr(trainer, 'all_params', getattr(base_trainer, 'all_params', None))
            if params is not None:
                # Count parameters
                total_params = 0
                param_info = {}
                for key, value in params.items():
                    if hasattr(value, 'shape'):
                        param_count = np.prod(value.shape)
                        param_info[key] = {
                            'shape': value.shape,
                            'count': param_count,
                            'mean': float(np.mean(value)),
                            'std': float(np.std(value))
                        }
                        total_params += param_count
                
                print(f"   Total parameters: {total_params}")
                print("   Parameter statistics:")
                for key, info in param_info.items():
                    print(f"     {key}: shape={info['shape']}, mean={info['mean']:.6f}, std={info['std']:.6f}")
                    
                    # Check for problematic values
                    if info['std'] < 1e-8:
                        print(f"     ⚠️ WARNING: {key} has very low variance - possible initialization issue")
                    if abs(info['mean']) > 10:
                        print(f"     ⚠️ WARNING: {key} has large mean values - possible exploding gradients")
            else:
                print("   ❌ No parameters found")
        
        # 4. Test on simple points
        print("\n4. PREDICTION TEST:")
        test_points = np.array([[0.5, 0.5], [0.0, 0.0], [1.0, 1.0], [0.25, 0.75]])
        
        try:
            if jax_available:
                test_input = jnp.array(test_points)
            else:
                test_input = test_points
                
            predictions = trainer.predict(test_input)
            
            if hasattr(predictions, 'numpy'):
                predictions = predictions.numpy()
            elif jax_available and hasattr(predictions, '__array__'):
                predictions = np.array(predictions)
            
            print(f"   Test predictions shape: {predictions.shape}")
            print(f"   Sample predictions:")
            for i, (point, pred) in enumerate(zip(test_points, predictions)):
                print(f"     Point {point}: ux={pred[0]:.6f}, uy={pred[1]:.6f}, p={pred[2]:.6f}")
            
            # Check for problematic predictions
            if np.any(np.isnan(predictions)):
                print("   ❌ CRITICAL: NaN values in predictions!")
            elif np.any(np.isinf(predictions)):
                print("   ❌ CRITICAL: Infinite values in predictions!")
            elif np.allclose(predictions, 0.0, atol=1e-10):
                print("   ⚠️ WARNING: All predictions are essentially zero - model not learning")
            elif np.std(predictions) < 1e-8:
                print("   ⚠️ WARNING: Very low prediction variance - model might be stuck")
            else:
                print("   ✅ Predictions seem reasonable")
                
        except Exception as e:
            print(f"   ❌ ERROR in prediction test: {e}")
    
    # 5. Physics equations test
    print("\n5. PHYSICS EQUATIONS TEST:")
    try:
        # Test if we can evaluate the physics residuals
        if hasattr(trainer, 'trainer') and hasattr(trainer.trainer, 'c'):
            problem = trainer.trainer.c.problem
            
            # Test physics evaluation on a small grid
            x_test = np.linspace(0.1, 0.9, 5)
            y_test = np.linspace(0.1, 0.9, 5)
            xx, yy = np.meshgrid(x_test, y_test)
            test_grid = np.column_stack([xx.flatten(), yy.flatten()])
            
            if jax_available:
                test_grid_jax = jnp.array(test_grid)
            else:
                test_grid_jax = test_grid
            
            # Try to evaluate physics residuals
            if hasattr(problem, 'physics_residual') or hasattr(problem, 'pde_residual'):
                print("   ✅ Physics residual function available")
                
                # Get current predictions
                current_pred = trainer.predict(test_grid_jax)
                
                # Calculate residuals (this tests if physics are working)
                if hasattr(problem, 'physics_residual'):
                    residuals = problem.physics_residual(trainer.all_params if hasattr(trainer, 'all_params') else trainer.trainer.all_params, test_grid_jax)
                else:
                    residuals = problem.pde_residual(trainer.all_params if hasattr(trainer, 'all_params') else trainer.trainer.all_params, test_grid_jax)
                
                if hasattr(residuals, 'numpy'):
                    residuals = residuals.numpy()
                elif jax_available and hasattr(residuals, '__array__'):
                    residuals = np.array(residuals)
                
                residual_norm = np.mean(np.abs(residuals))
                print(f"   Average physics residual: {residual_norm:.6f}")
                
                if residual_norm > 1.0:
                    print("   ⚠️ WARNING: Large physics residuals - model not satisfying equations well")
                elif residual_norm < 1e-6:
                    print("   ✅ Excellent physics satisfaction")
                else:
                    print("   ✅ Reasonable physics satisfaction")
                    
            else:
                print("   ⚠️ No physics residual function found")
                
        else:
            print("   ❌ Cannot access physics problem")
            
    except Exception as e:
        print(f"   ❌ ERROR in physics test: {e}")
        import traceback
        traceback.print_exc()
    
    # 6. Recommendations
    print("\n6. 🎯 RECOMMENDATIONS:")
    
    # Check the latest loss
    final_loss = None
    try:
        if hasattr(trainer, 'trainer') and hasattr(trainer.trainer, 'test_loss'):
            final_loss = trainer.trainer.test_loss()
    except:
        try:
            if hasattr(trainer, 'get_test_loss'):
                final_loss = trainer.get_test_loss()
        except:
            pass
    
    if final_loss is not None:
        print(f"   Current test loss: {final_loss:.6e}")
        
        if final_loss > 1e-1:
            print("   🔧 URGENT: Very high loss - try these fixes:")
            print("      - Increase training epochs (use comprehensive_training)")
            print("      - Check learning rate (try lower values like 1e-4)")
            print("      - Verify boundary conditions are correct")
            print("      - Check if problem setup matches physics")
        elif final_loss > 1e-3:
            print("   🔧 Moderate loss - try these improvements:")
            print("      - Run longer training with more epochs")
            print("      - Adjust loss weights (w_mech, w_flow, w_bc)")
            print("      - Try adaptive learning rate scheduling")
        else:
            print("   ✅ Loss looks reasonable")
    
    print("   📋 General recommendations:")
    print("      1. Run comprehensive_training() for better results")
    print("      2. Try different network architectures")
    print("      3. Experiment with loss weight balancing")
    print("      4. Check if exact solution is available for comparison")
    print("      5. Visualize training convergence over epochs")
    
    print("\n" + "="*60)

# Run diagnostics on the test trainer
if test_trainer is not None:
    comprehensive_physics_diagnostics(test_trainer)
else:
    print("❌ No trainer available for diagnostics")
    print("Run the quick physics test first")

## 🔧 Improved Training Solutions

Based on the diagnostics, here are better training approaches to fix the learning issues.

In [ ]:
def improved_physics_training():
    """Improved training with better parameters to fix learning issues"""
    if not module_status.get('physics', False):
        print("❌ ERROR: Physics trainer not available")
        return None
    
    print("🚀 STARTING IMPROVED PHYSICS TRAINING")
    print("This uses better parameters to fix learning issues")
    print("="*60)
    
    try:
        # Strategy 1: More training points and better architecture
        print("\n📈 Creating trainer with improved parameters...")
        trainer = BiotCoupledTrainer(
            # Better spatial resolution
            w_mech=1.0,          # Mechanics weight
            w_flow=1.0,          # Flow weight  
            w_bc=10.0,           # Higher boundary condition weight (important!)
            auto_balance=True,   # Auto balance losses
            
            # If these parameters exist, use them for better training
            # m_data_train=32,   # More training points (if supported)
            # n_epochs=500,      # More epochs (if supported)
            # verbose=True       # Verbose output (if supported)
        )
        
        print("✅ Improved trainer created")
        
        # Strategy 2: Staged training approach
        print("\n🎯 Starting staged training approach...")
        
        # Stage 1: Pre-training with focus on boundaries
        print("   Stage 1: Boundary-focused pre-training (50 steps)...")
        trainer.train_gradual_coupling(n_steps_pre=50, n_steps_coupled=0)
        
        # Check intermediate progress
        try:
            intermediate_loss = trainer.trainer.test_loss()
            print(f"   After pre-training: loss = {intermediate_loss:.6e}")
        except:
            print("   Pre-training completed")
        
        # Stage 2: Gradual coupling with more steps
        print("   Stage 2: Extended gradual coupling (200 steps)...")
        trainer.train_gradual_coupling(n_steps_pre=0, n_steps_coupled=200)
        
        # Final check
        try:
            final_loss = trainer.trainer.test_loss()
            print(f"✅ IMPROVED TRAINING COMPLETED")
            print(f"   Final loss: {final_loss:.6e}")
            
            if final_loss < 1e-2:
                print("   🎉 Excellent convergence!")
            elif final_loss < 1e-1:
                print("   ✅ Good convergence")
            else:
                print("   ⚠️ May need more training")
                
        except:
            print("✅ Training completed successfully!")
        
        return trainer
        
    except Exception as e:
        print(f"❌ ERROR in improved training: {e}")
        import traceback
        traceback.print_exc()
        return None

def alternative_training_approaches():
    """Alternative training strategies if the improved approach doesn't work"""
    print("🔬 ALTERNATIVE TRAINING APPROACHES")
    print("="*50)
    
    approaches = [
        {
            "name": "High Boundary Weight",
            "params": {"w_mech": 1.0, "w_flow": 1.0, "w_bc": 50.0, "auto_balance": False},
            "description": "Emphasizes boundary condition satisfaction"
        },
        {
            "name": "Balanced Auto-scaling", 
            "params": {"w_mech": 0.1, "w_flow": 0.1, "w_bc": 1.0, "auto_balance": True},
            "description": "Lets auto-balancing handle weight optimization"
        },
        {
            "name": "Flow-focused",
            "params": {"w_mech": 0.5, "w_flow": 2.0, "w_bc": 5.0, "auto_balance": True},
            "description": "Emphasizes fluid flow physics"
        }
    ]
    
    print("Try these parameter combinations:")
    for i, approach in enumerate(approaches, 1):
        print(f"\n{i}. {approach['name']}:")
        print(f"   Description: {approach['description']}")
        print(f"   Parameters: {approach['params']}")
        print(f"   Code: BiotCoupledTrainer(**{approach['params']})")
    
    print(f"\n💡 Usage example:")
    print(f"trainer = BiotCoupledTrainer(w_mech=1.0, w_flow=1.0, w_bc=50.0, auto_balance=False)")
    print(f"trainer.train_gradual_coupling(n_steps_pre=100, n_steps_coupled=300)")

def quick_comparison_test():
    """Quick test to compare different approaches"""
    if not module_status.get('physics', False):
        print("❌ Physics trainer not available")
        return
    
    print("⚡ QUICK COMPARISON TEST")
    print("Testing multiple approaches quickly...")
    print("="*40)
    
    test_configs = [
        {"name": "Original", "w_mech": 1.0, "w_flow": 1.0, "w_bc": 1.0},
        {"name": "High BC", "w_mech": 1.0, "w_flow": 1.0, "w_bc": 10.0},
        {"name": "Very High BC", "w_mech": 1.0, "w_flow": 1.0, "w_bc": 50.0}
    ]
    
    results = []
    
    for config in test_configs:
        print(f"\n🧪 Testing {config['name']} configuration...")
        try:
            trainer = BiotCoupledTrainer(
                w_mech=config['w_mech'],
                w_flow=config['w_flow'], 
                w_bc=config['w_bc'],
                auto_balance=True
            )
            
            # Quick training
            trainer.train_gradual_coupling(n_steps_pre=20, n_steps_coupled=30)
            
            # Test prediction quality
            test_point = np.array([[0.5, 0.5]])
            if jax_available:
                test_point = jnp.array(test_point)
            
            pred = trainer.predict(test_point)
            if hasattr(pred, 'numpy'):
                pred = pred.numpy()
            elif jax_available and hasattr(pred, '__array__'):
                pred = np.array(pred)
            
            # Get loss if possible
            try:
                loss = trainer.trainer.test_loss()
                loss_str = f"{loss:.2e}"
            except:
                loss_str = "N/A"
            
            results.append({
                'name': config['name'],
                'loss': loss_str,
                'prediction': pred.flatten() if pred is not None else None
            })
            
            print(f"   Loss: {loss_str}")
            print(f"   Sample prediction: {pred.flatten() if pred is not None else 'N/A'}")
            
        except Exception as e:
            print(f"   ❌ Failed: {e}")
            results.append({'name': config['name'], 'loss': 'Failed', 'prediction': None})
    
    print(f"\n📊 COMPARISON SUMMARY:")
    print(f"{'Config':<15} {'Loss':<15} {'Sample Prediction'}")
    print("-" * 50)
    for result in results:
        pred_str = str(result['prediction'][:3] if result['prediction'] is not None else 'N/A')
        print(f"{result['name']:<15} {result['loss']:<15} {pred_str}")
    
    return results

# Run the quick comparison first
print("🔍 Let's first do a quick comparison of different approaches:")
comparison_results = quick_comparison_test()

## Training Loss Visualization

Plot the training history if available.

In [ ]:
def plot_training_history(trainer):
    """Plot training loss history"""
    if trainer is None:
        print("No trainer provided")
        return
    
    if not plotting_available:
        print("Matplotlib not available for plotting")
        return
    
    try:
        # Get loss history
        if hasattr(trainer, 'loss_history'):
            losses = trainer.loss_history
        elif hasattr(trainer, 'trainer') and hasattr(trainer.trainer, 'loss_history'):
            losses = trainer.trainer.loss_history
        else:
            print("No loss history available")
            return
        
        if len(losses) == 0:
            print("Empty loss history")
            return
        
        plt.figure(figsize=(10, 6))
        plt.semilogy(losses)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss History')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"\nTraining Summary:")
        print(f"Total epochs: {len(losses)}")
        print(f"Initial loss: {losses[0]:.6f}")
        print(f"Final loss: {losses[-1]:.6f}")
        print(f"Loss reduction: {losses[0]/losses[-1]:.2f}x")
        
    except Exception as e:
        print(f"Error plotting training history: {e}")

# Plot training history for the test trainer
if test_trainer is not None:
    plot_training_history(test_trainer)
else:
    print("No trainer available - run the physics test first")

## Comprehensive Training (Optional)

Run a more comprehensive training session with better parameters.

In [ ]:
def comprehensive_training():
    """Run comprehensive training with better parameters"""
    if not module_status.get('physics', False):
        print("ERROR: Physics trainer not available")
        return None
    
    print("Starting comprehensive training...")
    print("This may take several minutes")
    
    try:
        # Create trainer with better settings
        trainer = BiotCoupledTrainer(
            m_data_train=16,      # More training points
            m_subdomain_n=2,      # Multiple subdomains
            l_data_train=3,       # More boundary points
            n_epochs=1000,        # More training epochs
            verbose=True
        )
        
        print("SUCCESS: Comprehensive trainer created")
        
        # Training
        print("Running comprehensive training (1000 epochs)...")
        trainer.train()
        
        print("SUCCESS: Comprehensive training completed")
        
        # Get final loss
        final_loss = trainer.get_test_loss()
        print(f"Final test loss: {final_loss:.6f}")
        
        return trainer
        
    except Exception as e:
        print(f"ERROR in comprehensive training: {e}")
        return None

# Uncomment the line below to run comprehensive training
# comprehensive_trainer = comprehensive_training()

print("To run comprehensive training, uncomment the line above")
print("This will take significantly longer but produce better results")

## Data-Enhanced Training (Optional)

If VTK data is available, test the data-enhanced trainer.

In [ ]:
def test_data_enhanced_training():
    """Test data-enhanced training if available"""
    if not module_status.get('data', False):
        print("Data-enhanced trainer not available")
        return None
    
    # Check if data directory exists
    data_dir = Path("../Data_2D")
    if not data_dir.exists():
        print(f"Data directory not found: {data_dir}")
        print("Data-enhanced training requires VTK files in Data_2D/")
        return None
    
    print("Testing data-enhanced training...")
    
    try:
        # Create data loader
        data_loader = VTKDataLoader(str(data_dir))
        
        # List available files
        files = data_loader.list_available_files()
        print(f"Found {len(files)} VTK files")
        
        # Create data-enhanced trainer
        trainer = BiotCoupledDataTrainer(
            data_loader=data_loader,
            m_data_train=8,
            n_epochs=100,
            verbose=True
        )
        
        print("SUCCESS: Data-enhanced trainer created")
        
        # Quick training
        trainer.train()
        
        print("SUCCESS: Data-enhanced training completed")
        
        return trainer
        
    except Exception as e:
        print(f"ERROR in data-enhanced training: {e}")
        return None

# Test data-enhanced training
data_trainer = test_data_enhanced_training()

## Summary and Next Steps

Summarize what we've accomplished and suggest next steps.

In [ ]:
def print_summary():
    """Print a summary of what we've accomplished"""
    print("\n" + "="*60)
    print("BIOT POROELASTICITY VISUALIZATION SUMMARY")
    print("="*60)
    
    print("\nMODULE STATUS:")
    for module, status in module_status.items():
        print(f"  {module}: {'Available' if status else 'Not available'}")
    
    print("\nLIBRARY STATUS:")
    print(f"  JAX: {'Available' if jax_available else 'Not available (using NumPy)'}")
    print(f"  Plotting: {'Available' if plotting_available else 'Not available'}")
    
    print("\nTRAINER STATUS:")
    print(f"  Quick test: {'Completed' if test_trainer is not None else 'Not run'}")
    print(f"  Data-enhanced: {'Available' if data_trainer is not None else 'Not available'}")
    
    if test_trainer is not None:
        try:
            final_loss = test_trainer.trainer.test_loss()
            print(f"\nQUICK TEST RESULTS:")
            print(f"  Final loss: {final_loss:.6f}")
            if final_loss < 1e-2:
                print(f"  Quality: Excellent (< 1e-2)")
            elif final_loss < 1e-1:
                print(f"  Quality: Good (< 1e-1)")
            else:
                print(f"  Quality: Needs more training (> 1e-1)")
        except:
            print(f"\nQUICK TEST RESULTS:")
            print(f"  Training completed successfully")
            print(f"  Quality: Ready for visualization")
    
    print("\nNEXT STEPS:")
    print("  1. Run comprehensive training for better accuracy")
    print("  2. Experiment with different parameters")
    print("  3. Add experimental data if available")
    print("  4. Explore different visualization options")
    
    print("\nSUCCESS: Biot poroelasticity visualization is working!")
    print("="*60)

# Print the summary
print_summary()